<h2>Desarrollo del Desafio 2 - Machine Learning


<h3>Grupo 7

<li>Laura Jazmín Chao
<li>Juan Nicolás Capistrano
<li>Betiana Srur
<li>Ignacio Mendieta
<li>Marecelo Carrizo

**Desafio 2** : Aplicar las tecnicas de Machine Learning para calcular el precio por metro cuadrado de las propiedades.

In [38]:
import numpy as np
import pandas as pd

# Libreria Grafica.
%matplotlib inline
import seaborn as sns
from matplotlib import pyplot as plt

# Libreria para analisis ML
from sklearn import datasets, linear_model

In [39]:
# Definimos la ruta de la información.
data_propiedades = "Data/properati.csv"

# Leemos los datos del archivo
data = pd.read_csv(data_propiedades, sep=",", encoding="UTF-8")

# Chequeamos que los datos se hayan importado correctamente
data.sample(5)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
100289,100289,sell,apartment,Córdoba,|Argentina|Córdoba|Córdoba|,Argentina,Córdoba,3860259.0,NaN,NaN,...,251.0,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/1bnqk_venta_depart...,"- PISO COMPLETO - Consta de recepción, living ...",Departamento en Centro,https://thumbs4.properati.com/6/11Ht69VKGLvgns...
108568,108568,sell,house,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|La ...,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,NaN,NaN,...,200.0,NaN,900.000000,NaN,NaN,NaN,http://www.properati.com.ar/1c0ir_venta_casa_l...,CASA PH AL FRENTE CON CONSULTORIO CON ENTRADA ...,Casa - La Plata,https://thumbs4.properati.com/1/3SurKvpdAhKnhS...
27275,27275,sell,apartment,Barracas,|Argentina|Capital Federal|Barracas|,Argentina,Capital Federal,3436134.0,"-34.631255,-58.370463",-34.631255,...,58.0,NaN,1655.810345,3.0,3.0,NaN,http://www.properati.com.ar/17t45_venta_depart...,Corredor Responsable: Jorge Salafia - CUCICBA ...,VENTA 2 AMB PATRICIOS 100 FINANC.h/ 60 CUOTAS,https://thumbs4.properati.com/9/ZZnIFTIw0uaMGG...
49532,49532,sell,apartment,Valentín Alsina,|Argentina|Bs.As. G.B.A. Zona Sur|Lanús|Valent...,Argentina,Bs.As. G.B.A. Zona Sur,NaN,NaN,NaN,...,NaN,1818.181818,NaN,1.0,2.0,NaN,http://www.properati.com.ar/19ccb_venta_depart...,Venta de Departamento 2 AMBIENTES en VALENTIN ...,DEPARTAMENTO EN VENTA,https://thumbs4.properati.com/8/V_xpOHWYDvack7...
74515,74515,sell,apartment,Nuñez,|Argentina|Capital Federal|Nuñez|,Argentina,Capital Federal,3430348.0,NaN,NaN,...,53.0,2382.716049,3641.509434,NaN,NaN,NaN,http://www.properati.com.ar/1ap3d_venta_depart...,ESPECIFICACIONES DE LOS DEPARTAMENTOSEstructu...,Departamento - Nuñez,https://thumbs4.properati.com/2/rYdgQ_1UjuUnKj...


In [40]:
# Chequeamos cantidad de registros y cantidad de columnas
data.shape

(121220, 26)

<h2>Limpieza de Datos.

In [41]:
# Creamos una mask para traer los valores en monedas que no son USD ni ARS.
currency_OTHER_CURRENCY_mask = (data.currency == 'PEN') | (data.currency == 'UYU')
# Observamos esos registros
data.loc[currency_OTHER_CURRENCY_mask, :]

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
50387,50387,sell,apartment,Capital Federal,|Argentina|Capital Federal|,Argentina,Capital Federal,3433955.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/19e8z_venta_depart...,Ref.: 55:PREVENTA ::: OPORTUNIDAD ::: Dúplex a...,DUPLEX A LA VENTA | Liniers 1200 | Tandil,https://thumbs4.properati.com/9/N_Qr_5X_vOq--b...
50388,50388,sell,apartment,Capital Federal,|Argentina|Capital Federal|,Argentina,Capital Federal,3433955.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/19e90_venta_depart...,Ref.: 56Duplex en venta de 1 dormitorio con ap...,DUPLEX A LA VENTA | Liniers 1200 | Tandil,https://thumbs4.properati.com/8/_cBXAyr_9pDySg...
107390,107390,sell,house,San Roque,|Argentina|Mendoza|San Roque|,Argentina,Mendoza,3836586.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/1by0s_venta_casa_s...,"Casa con Calefacción central, Mendoza, San Roq...",Casa - San Vicente,https://thumbs4.properati.com/2/pA0RbuDKRK6GgV...


In [42]:
# Hacemos drop de esos registros y chequeamos la diferencia entre cantidad original y actual
cant_registros = data.shape[0]
data.drop(data.loc[currency_OTHER_CURRENCY_mask, :].index, inplace = True)
print(f'Cantidad de registros original - registro actuales: {cant_registros - data.shape[0]}')

Cantidad de registros original - registro actuales: 3


In [43]:
# Creamos la lista de columnas a filtrar
drop_columns = ['properati_url', 'Unnamed: 0', 'operation','image_thumbnail','expenses','lat-lon','geonames_id','lat', 'lon']

# Creamos una variable nueva para guardar los datos (el cambio se realiza inplace=False)
data_filtered = data.drop(drop_columns, axis=1)
data_filtered.head(5)

,property_type,place_name,place_with_parent_names,country_name,state_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,description,title
0,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,62000.0,USD,1093959.0,62000.0,55.0,40.0,1127.272727,1550.000000,NaN,NaN,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB
1,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,150000.0,USD,2646675.0,150000.0,NaN,NaN,NaN,NaN,NaN,NaN,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...
2,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,72000.0,USD,1270404.0,72000.0,55.0,55.0,1309.090909,1309.090909,NaN,NaN,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO
3,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,95000.0,USD,1676227.5,95000.0,NaN,NaN,NaN,NaN,NaN,NaN,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado
4,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,64000.0,USD,1129248.0,64000.0,35.0,35.0,1828.571429,1828.571429,NaN,NaN,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...


**Esto no es necesario ahora ¿?**

In [44]:
# Creamos un diccionario con la nueva agrupación
provincia={'Capital Federal':'CABA','Bs.As. G.B.A. Zona Norte': 'Bs. As. G.B.A.', 'Bs.As. G.B.A. Zona Sur':'Bs. As. G.B.A.', 'Córdoba':'Córdoba','Santa Fe':'Santa Fe','Buenos Aires Costa Atlántica':'Buenos Aires', 'Bs.As. G.B.A. Zona Oeste':'Bs. As. G.B.A.', 'Buenos Aires Interior':'Buenos Aires','Río Negro':'Resto del país','Neuquén':'Resto del país', 'Mendoza':'Resto del país', 'Tucumán':'Resto del país', 'Corrientes':'Resto del país', 'Misiones':'Resto del país', 'Entre Ríos':'Resto del país', 'Salta':'Resto del país', 'Chubut':'Resto del país', 'San Luis':'Resto del país', 'La Pampa':'Resto del país','Formosa':'Resto del país','Chaco': 'Resto del país', 'San Juan':'Resto del país','Tierra Del Fuego':'Resto del país', 'Catamarca':'Resto del país', 'Jujuy': 'Resto del país', 'Santa Cruz':'Resto del país', 'La Rioja':'Resto del país', 'Santiago Del Estero':'Resto del país'}

In [45]:
# Mapeamos las provincias con el diccionario y ponermos el grupo en una columna nueva
data_filtered['province_group'] = data_filtered.state_name.map(provincia)
display(data_filtered.sample(5))


,property_type,place_name,place_with_parent_names,country_name,state_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,description,title,province_group
108571,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|La ...,Argentina,Bs.As. G.B.A. Zona Sur,110000.0,USD,1940895.00,110000.0,NaN,72.0,NaN,1527.777778,NaN,NaN,MUY BUEN DPTO RECICLADO. AMPLIA COCHERA CUBIER...,Departamento - La Plata,Bs. As. G.B.A.
59405,apartment,San Fernando,|Argentina|Bs.As. G.B.A. Zona Norte|San Fernan...,Argentina,Bs.As. G.B.A. Zona Norte,107000.0,USD,1887961.50,107000.0,55.0,50.0,1945.454545,2140.000000,NaN,2.0,Depto 2 Ambientes con balcón a EstrenarENTREGA...,Constitución al 2200 Dpto 2 Ambientes A Estren...,Bs. As. G.B.A.
48838,apartment,San Telmo,|Argentina|Capital Federal|San Telmo|,Argentina,Capital Federal,111100.0,USD,1960303.95,111100.0,45.0,38.0,2468.888889,2923.684211,NaN,1.0,Descripción General: Ubicado a metros del Cora...,Nuevo Edificio en San Telmo,CABA
54361,apartment,Tigre,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre|,Argentina,Bs.As. G.B.A. Zona Norte,270000.0,USD,4764015.00,270000.0,370.0,370.0,729.729730,729.729730,NaN,4.0,***AMPLIO PARQUE | LOTE INTERNO | EXCELENTE CA...,Bustamante Propiedades | Los Olmos en Venta,Bs. As. G.B.A.
18669,apartment,Monte Hermoso,|Argentina|Buenos Aires Costa Atlántica|Monte ...,Argentina,Buenos Aires Costa Atlántica,100000.0,USD,1764450.00,100000.0,42.0,NaN,2380.952381,NaN,NaN,2.0,"DEPARTAMENTO A ESTRENAR DE 1 DORMITORIO, COCIN...",DEPARTAMENTO DE 1 DORMITORIO EN EDIFICIO FRENT...,Buenos Aires


<h2>Tomamos solamente CABA para el analisis.

In [46]:
# Creamos una máscara y la aplicamos al dataframe anterior para traer los registros que necesitamos
data_caba_mask = data_filtered.state_name == 'Capital Federal' 
data_caba = data_filtered.loc[data_caba_mask, :]

# Dropeamos la columna creada anteriormente.
data_caba.drop(columns=['province_group', 'country_name', 'state_name', 'place_with_parent_names','description', 'title'], inplace=True)

# Chequeamos cómo quedaron los datos
data_caba.head(3)

,property_type,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms
0,PH,Mataderos,62000.0,USD,1093959.0,62000.0,55.0,40.0,1127.272727,1550.000000,NaN,NaN
2,apartment,Mataderos,72000.0,USD,1270404.0,72000.0,55.0,55.0,1309.090909,1309.090909,NaN,NaN
3,PH,Liniers,95000.0,USD,1676227.5,95000.0,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
# Vemos la cantidad de datos solamente de CABA.
data_caba.shape

(32314, 12)

In [48]:
# Revisamos si hay barrios de caba mal indentificados "Ejemplo: Capital Federal"
pd.set_option('display.max_rows', 1000) # Para mostrar todas las filas.


In [49]:
# Dropeamos los de Capital Federal
capi_mask = data_caba.place_name == 'Capital Federal'
data_caba.drop(data_caba.loc[capi_mask, :].index, inplace=True)

In [50]:
data_caba.shape

(31019, 12)

In [51]:
data_caba.head()

,property_type,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms
0,PH,Mataderos,62000.0,USD,1093959.0,62000.0,55.0,40.0,1127.272727,1550.000000,NaN,NaN
2,apartment,Mataderos,72000.0,USD,1270404.0,72000.0,55.0,55.0,1309.090909,1309.090909,NaN,NaN
3,PH,Liniers,95000.0,USD,1676227.5,95000.0,NaN,NaN,NaN,NaN,NaN,NaN
7,apartment,Belgrano,138000.0,USD,2434941.0,138000.0,45.0,40.0,3066.666667,3450.000000,NaN,NaN
8,apartment,Belgrano,195000.0,USD,3440677.5,195000.0,65.0,60.0,3000.000000,3250.000000,NaN,NaN


## De los datos filtrados según Desafio 1 deberiamos:

1. Terminar de dropear variables incompletas que serian necesarias para poder entrenar un modelo. (ejemplo: si la propiedad no posee: price, price_aprox_local_currency,price_usd_per_m2, price_per_m2, etc...) la información de los barrios ya fue analizado y dropeado en el paso anterior.
2. Definir las variables CUANTITATIVAS, ejemplo precio, total de m2 de superficie.
3. Armar las variables CUALITATICAS, Dummy (ejemplo: place_name, property_type).
4. Armar una nueva variable Dummy que contemple los precios de propiedad segun su distribución por metro cuadrado. Utilizando Panda CUT. 

## Finalizado el analisis de datos, armamos diferentes modelos para predecir el precio de la propiedad.

1. Modelo No Supervisado (Regresion Lineal Multiple)
2. Entrenar los modelos con Lasso, RidgeCV y utilizando Cross Validation.



In [55]:
data_caba.

,property_type,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms
691,PH,Belgrano,152000.0,USD,2681964.0,152000.0,52.0,52.0,2923.076923,2923.076923,6.0,2.0
692,PH,Belgrano,152000.0,USD,2681964.0,152000.0,52.0,52.0,2923.076923,2923.076923,6.0,3.0
693,PH,Belgrano,152000.0,USD,2681964.0,152000.0,52.0,52.0,2923.076923,2923.076923,6.0,3.0
694,PH,Coghlan,152000.0,USD,2681964.0,152000.0,52.0,52.0,2923.076923,2923.076923,6.0,2.0
695,PH,Coghlan,152000.0,USD,2681964.0,152000.0,52.0,52.0,2923.076923,2923.076923,6.0,3.0
